## Analysis of business opportunities near Metro stations in Delhi

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import requests # library to handle requests
from lxml import html # library to read html page
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.37 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.18 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  35.87 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.14 MB/s
Libraries imported.


## 1. Download and Explore Dataset

In [16]:
#Fetch metro stations list from wikipedia page.
r = requests.get("https://en.wikipedia.org/wiki/List_of_Delhi_Metro_stations")
soup = BeautifulSoup(r.content, 'html.parser')
data = []
table = soup.find('table', class_="wikitable sortable")
table_body = table.find('tbody')
rows = table_body.find_all('tr')

for row in rows:    
    cols = row.find_all(['td','th'])
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# Create dataframe
df = pd.DataFrame(data)
df = df[1:]

# Define column header
df.columns = ['Sr','StationName','Hindi Name','Line','Opened','Layout','Notes','Refs','Platform Layout']

# Ignore additional columns
df=df[['StationName','Line','Opened','Layout']]

# Ignore cells with metro lines name as station name
df=df[~df.StationName.str.contains("Line")]

# Ignore cells with station layout 'At Grade', as these stations are for interchange only and have no exit points.
df=df[~df.Layout.str.contains("Grade")]

# Remove special symbols from column names.
df['StationName']=df['StationName'].str.replace('*','')
df['StationName']=df['StationName'].str.replace('†','')
df['StationName']=df['StationName'].str.replace('¤','')

# Converting station opening date to datetime formt.
df['Opened'] = pd.to_datetime(df['Opened'])

df.reset_index(drop=True)

,StationName,Line,Opened,Layout
0,Adarsh Nagar,Yellow Line,2009-02-04,Elevated
1,AIIMS,Yellow Line,2010-09-03,Underground
2,Akshardham,Blue Line,2009-11-12,Elevated
3,Anand Vihar ISBT,Blue Line branch,2010-01-06,Elevated
4,Arjan Garh,Yellow Line,2010-06-21,Elevated
5,Arthala,Red Line,2019-03-08,Elevated
6,Ashok Park Main,Green Line,2010-04-02,Elevated
7,Ashram,Pink Line,2018-12-31,Underground
8,Azadpur,Yellow Line,2009-02-04,Elevated
9,Badarpur Border,Violet Line,2011-01-14,Elevated


#### Let's create a function to get latitude, longitude of each cell

In [17]:
def getLatiLong(row):
    address=row['StationName']
    add=''
    latitude = 0
    longitude = 0
    geolocator = Nominatim(user_agent="ny_explorer")
    try:
        add = address+', National Capital Territory of Delhi, IN'
        location = geolocator.geocode(add)
        latitude = location.latitude
        longitude = location.longitude
    except:
        if '–' in address:
            add = address.split('–')[0]
            add+=', National Capital Territory of Delhi, IN'
        else:  
            add = address+', IN'
        try:
            location = geolocator.geocode(add)
            latitude = location.latitude
            longitude = location.longitude 
        except:
            latitude = 0
            longitude = 0
            
    return pd.Series([latitude,longitude])

In [2]:
df[['Latitude','Longitude']] = df.apply(getLatiLong, axis=1)

NameError: name 'df' is not defined

In [3]:
df = df[df.Latitude != 0.000000]

NameError: name 'df' is not defined

In [20]:
df.reset_index(drop=True)

,StationName,Line,Opened,Layout,Latitude,Longitude
0,Adarsh Nagar,Yellow Line,2009-02-04,Elevated,28.614193,77.071541
1,AIIMS,Yellow Line,2010-09-03,Underground,28.569016,77.207612
2,Akshardham,Blue Line,2009-11-12,Elevated,28.612517,77.277318
3,Anand Vihar ISBT,Blue Line branch,2010-01-06,Elevated,28.646702,77.315509
4,Arjan Garh,Yellow Line,2010-06-21,Elevated,28.480716,77.125784
5,Ashok Park Main,Green Line,2010-04-02,Elevated,28.671633,77.155301
6,Ashram,Pink Line,2018-12-31,Underground,28.575177,77.256932
7,Azadpur,Yellow Line,2009-02-04,Elevated,28.707069,77.180383
8,Badarpur Border,Violet Line,2011-01-14,Elevated,28.493416,77.303334
9,Bahadurgarh City,Green Line,2018-06-24,Elevated,28.693324,76.933237


#### Save final processed data as DelhiMetroData.csv file. 

In [21]:
# The code was removed by Watson Studio for sharing.

{'asset_id': 'b32ec158-1b7e-466b-9d78-44cfafcbd5f9',
 'bucket_name': 'advancedatascience-donotdelete-pr-w2gymqcfyoewwc',
 'file_name': 'DelhiMetroData.csv',
 'message': 'File saved to project storage.'}

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
df.head()

,StationName,Line,Opened,Layout,Latitude,Longitude
0,Adarsh Nagar,Yellow Line,2009-02-04,Elevated,28.614192,77.071541
1,AIIMS,Yellow Line,2010-09-03,Underground,28.569016,77.207612
2,Akshardham,Blue Line,2009-11-12,Elevated,28.612517,77.277318
3,Anand Vihar ISBT,Blue Line branch,2010-01-06,Elevated,28.646702,77.315509
4,Arjan Garh,Yellow Line,2010-06-21,Elevated,28.480716,77.125784


#### Use geopy library to get the latitude and longitude values of National Capital Territory of Delhi, India.

In [4]:
address = 'National Capital Territory of Delhi, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of National Capital Territory of Delhi, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of National Capital Territory of Delhi, India are 28.6273928, 77.1716954.


#### Create a map of Delhi Metro with stations superimposed on top.

In [5]:
# create map of National Capital Territory of Delhi using latitude and longitude values
map_nct = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, stationName in zip(df['Latitude'], df['Longitude'], df['StationName']):
    label = '{}'.format(stationName)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        zoom_start=100,
        parse_html=False).add_to(map_nct)  
    
map_nct

Next, we are going to start utilizing the Foursquare API to explore the metro stations and segment them.

#### Define Foursquare Credentials and Version

In [6]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the first metro station in our dataframe.

Get the station's name.

In [7]:
df.loc[0, 'StationName']

'Adarsh Nagar'

Get the station's latitude and longitude values.

In [8]:
station_latitude = df.loc[0, 'Latitude'] # station latitude value
station_longitude = df.loc[0, 'Longitude'] # station longitude value

station_name = df.loc[0, 'StationName'] # station name

print('Latitude and longitude values of {} are {}, {}.'.format(station_name, 
                                                               station_latitude, 
                                                               station_longitude))

Latitude and longitude values of Adarsh Nagar are 28.614192499999998, 77.0715411848447.


#### Now, let's get the top venues that are in Adarsh Nagarl within a radius of 500 meters.

Let's create the GET request URL.

In [9]:
# type your answer here
LIMIT = 100 
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, station_latitude, station_longitude, VERSION, radius, LIMIT)


Send the GET request and examine the results

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9e61a41ed219272ae2f5ee'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-586ceb469c439d4ddb887417-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
          'suffix': '.png'},
         'id': '5453de49498eade8af355881',
         'name': 'Business Service',
         'pluralName': 'Business Services',
         'primary': True,
         'shortName': 'Business Services'}],
       'id': '586ceb469c439d4ddb887417',
       'location': {'cc': 'IN',
        'country': 'India',
        'distance': 388,
        'formattedAddress': ['Illinois', 'India'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.617452804245467,
          'lng': 77.07295417785645}],
        'lat': 28.617452804245467,
        'lng': 7

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Get Top Rank,Business Service,28.617453,77.072954
1,Bikanerwala,Indian Restaurant,28.613391,77.076084


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## 2. Explore Metro Stations in NCT of Delhi

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['StationName', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each metro station and create a new dataframe called *metro_venues*.

In [15]:
metro_venues = getNearbyVenues(names=df['StationName'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Adarsh Nagar
AIIMS
Akshardham
Anand Vihar ISBT
Arjan Garh
Ashok Park Main
Ashram
Azadpur
Badarpur Border
Bahadurgarh City
Barakhambha Road
Bata Chowk
Bhikaji Cama Place
Botanical Garden
Central Secretariat
Chandni Chowk
Chawri Bazar
Chhatarpur
Chirag Delhi
Dabri Mor-Janakpuri South
Dashrath Puri
Delhi Aerocity
Delhi Cantonment
Delhi Gate
Dhaula Kuan
Dilshad Garden
Durgabai Deshmukh South Campus
Dwarka
Dwarka Mor
Dwarka Sector 8
Dwarka Sector 9
Dwarka Sector 10
Dwarka Sector 11
Dwarka Sector 12
Dwarka Sector 13
Dwarka Sector 14
Dwarka Sector 21
East Azad Nagar
East Vinod Nagar – Mayur Vihar-II
Escorts Mujesar
ESI Hospital
Ghitorni
Gokulpuri
Golf Course
Govind Puri
Greater Kailash
Green Park
GTB Nagar
Guru Dronacharya
Haiderpur
Hauz Khas
Hazrat Nizamuddin
Hindon
HUDA City Centre
IFFCO Chowk
IIT Delhi
INA
Inderlok
Indira Gandhi International Airport
Indraprastha
IP Extension
ITO
Jaffrabad
Jahangirpuri
Jama Masjid
Jamia Millia Islamia
Janakpuri East
Janakpuri West
Jangpura
Janpath
Jasola V

Let's check the size of the resulting dataframe

In [16]:
print(metro_venues.shape)
metro_venues.head()

(1488, 7)


,StationName,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614192,77.071541,Get Top Rank,28.617453,77.072954,Business Service
1,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
2,AIIMS,28.569016,77.207612,Dilli Haat | दिल्ली हाट (Dilli Haat),28.573132,77.208810,Market
3,AIIMS,28.569016,77.207612,Nagaland Stall,28.573070,77.206521,Restaurant
4,AIIMS,28.569016,77.207612,Navdanya Organic Cafe,28.572903,77.206451,Indian Restaurant


Let's check how many venues were returned for each metro station

In [17]:
metro_venue_count=metro_venues.groupby('StationName').count()
metro_venue_count=metro_venue_count[['Venue']].sort_values(by=['Venue'], ascending=False).reset_index()
metro_venue_count

,StationName,Venue
0,Lal Qila,96
1,Rajiv Chowk,81
2,Khan Market,47
3,Indira Gandhi International Airport,42
4,Noida Sector 18,40
5,Rajouri Garden,36
6,Nehru Place,28
7,Shivaji Stadium,27
8,Green Park,26
9,Delhi Aerocity,23


#### Let's find most common venue types

In [18]:
metro_venue_category_count=metro_venues.groupby('Venue Category').count()
metro_venue_category_count=metro_venue_category_count[['Venue']].sort_values(by=['Venue'], ascending=False).reset_index()
metro_venue_category_count

,Venue Category,Venue
0,Indian Restaurant,144
1,Hotel,88
2,Café,78
3,Fast Food Restaurant,68
4,Coffee Shop,67
5,Light Rail Station,55
6,Pizza Place,53
7,Chinese Restaurant,40
8,Restaurant,35
9,Train Station,33


#### Dropping venues which are metro stations itself.

In [87]:
metro_venue_category_count=metro_venue_category_count[~metro_venue_category_count['Venue Category'].str.contains("Station")]

#### Taking into consideration top 25 venue types.

In [88]:
metro_venue_category_count.reset_index(drop=True).head(25)

,Venue Category,Venue
0,Indian Restaurant,138
1,Hotel,83
2,Café,80
3,Coffee Shop,66
4,Fast Food Restaurant,65
5,Pizza Place,55
6,Chinese Restaurant,38
7,Restaurant,36
8,Shopping Mall,27
9,Bar,26


#### Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(metro_venues['Venue Category'].unique())))

There are 221 uniques categories.


## 3. Analyze Each Metro Station

In [21]:
# one hot encoding
metro_onehot = pd.get_dummies(metro_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
metro_onehot['StationName'] = metro_venues['StationName'] 

# move neighborhood column to the first column
fixed_columns = [metro_onehot.columns[-1]] + list(metro_onehot.columns[:-1])
metro_onehot = metro_onehot[fixed_columns]

metro_onehot.head()

,StationName,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Center,Business Service,Café,Campground,Candy Store,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Travel,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Lawyer,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Ramen Restaurant,Resort,Restaurant,River,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Souvenir Shop,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AIIMS,0,0,0,

And let's examine the new dataframe size.

In [22]:
metro_onehot.shape

(1488, 222)

#### Next, let's group rows by StationName and by taking the mean of the frequency of occurrence of each category

In [23]:
metro_grouped = metro_onehot.groupby('StationName').mean().reset_index()
metro_grouped

,StationName,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Center,Business Service,Café,Campground,Candy Store,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Travel,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Lawyer,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Ramen Restaurant,Resort,Restaurant,River,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Souvenir Shop,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,AIIMS,0.000000,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

Let's confirm the new size

In [24]:
metro_grouped.shape

(193, 222)

First, let's write a function to sort the venues in descending order.

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each station.

In [61]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['StationName']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
metro_venues_sorted = pd.DataFrame(columns=columns)
metro_venues_sorted['StationName'] = metro_grouped['StationName']

for ind in np.arange(metro_grouped.shape[0]):
    metro_venues_sorted.iloc[ind, 1:] = return_most_common_venues(metro_grouped.iloc[ind, :], num_top_venues)

metro_venues_sorted

,StationName,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,AIIMS,Indian Restaurant,Market,Snack Place,Restaurant,Memorial Site
1,Adarsh Nagar,Indian Restaurant,Business Service,Yoga Studio,Duty-free Shop,Food & Drink Shop
2,Akshardham,Hindu Temple,Athletics & Sports,Yoga Studio,Eastern European Restaurant,Food & Drink Shop
3,Anand Vihar ISBT,Multiplex,Clothing Store,Light Rail Station,Café,Shopping Mall
4,Arjan Garh,Hotel,Yoga Studio,Furniture / Home Store,Light Rail Station,Eastern European Restaurant
5,Ashok Park Main,Train Station,Yoga Studio,Duty-free Shop,Food & Drink Shop,Food
6,Ashram,Bakery,Neighborhood,Dessert Shop,Shopping Mall,Yoga Studio
7,Azadpur,Park,Bus Station,Indian Restaurant,Food,Flower Shop
8,Badarpur Border,ATM,Hotel,Train Station,Duty-free Shop,Food & Drink Shop
9,Bahadurgarh City,Bus Station,Train Station,Duty-free Shop,Food & Drink Shop,Food


## 4. Cluster Metro Stations

Run *k*-means to cluster the neighborhood into 5 clusters.

In [62]:
# set number of clusters
kclusters =5

metro_grouped_clustering = metro_grouped.drop('StationName', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(metro_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 4, 4, 4, 0, 4, 3, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each station.

In [63]:
# add clustering labels
metro_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
metro_merged = df

metro_merged = metro_merged.join(metro_venues_sorted.set_index('StationName'), on='StationName')

metro_merged.head() # check the last columns!

,StationName,Line,Opened,Layout,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adarsh Nagar,Yellow Line,2009-02-04,Elevated,28.614192,77.071541,3.0,Indian Restaurant,Business Service,Yoga Studio,Duty-free Shop,Food & Drink Shop
1,AIIMS,Yellow Line,2010-09-03,Underground,28.569016,77.207612,3.0,Indian Restaurant,Market,Snack Place,Restaurant,Memorial Site
2,Akshardham,Blue Line,2009-11-12,Elevated,28.612517,77.277318,4.0,Hindu Temple,Athletics & Sports,Yoga Studio,Eastern European Restaurant,Food & Drink Shop
3,Anand Vihar ISBT,Blue Line branch,2010-01-06,Elevated,28.646702,77.315509,4.0,Multiplex,Clothing Store,Light Rail Station,Café,Shopping Mall
4,Arjan Garh,Yellow Line,2010-06-21,Elevated,28.480716,77.125784,4.0,Hotel,Yoga Studio,Furniture / Home Store,Light Rail Station,Eastern European Restaurant


In [64]:
metro_merged['Cluster Labels'] = metro_merged['Cluster Labels'].fillna(0).astype(int).astype(int)

Finally, let's visualize the resulting clusters

In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(metro_merged['Latitude'], metro_merged['Longitude'], metro_merged['StationName'], metro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [86]:
cluster1=metro_merged.loc[metro_merged['Cluster Labels'] == 0, metro_merged.columns[[0] + list(range(2, metro_merged.shape[1]))]]
cluster1['3rd Most Common Venue'].dropna()
cluster1

,StationName,Opened,Layout,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Ashok Park Main,2010-04-02,Elevated,28.671633,77.155301,0,Train Station,Yoga Studio,Duty-free Shop,Food & Drink Shop,Food
8,Badarpur Border,2011-01-14,Elevated,28.493416,77.303334,0,ATM,Hotel,Train Station,Duty-free Shop,Food & Drink Shop
9,Bahadurgarh City,2018-06-24,Elevated,28.693324,76.933237,0,Bus Station,Train Station,Duty-free Shop,Food & Drink Shop,Food
30,Dwarka Sector 9,2006-04-01,Elevated,28.574282,77.065351,0,Herbs & Spices Store,Fast Food Restaurant,Train Station,Chaat Place,Burger Joint
43,Golf Course,2009-11-12,Elevated,28.597781,77.159817,0,Golf Course,Snack Place,Train Station,Sake Bar,Yoga Studio
44,Govind Puri,2010-10-03,Elevated,28.544377,77.264266,0,Beer Garden,Train Station,Pizza Place,Indian Restaurant,Park
50,Hauz Khas,2010-09-03,Underground,28.544258,77.206740,0,Lounge,Chinese Restaurant,Nightclub,Train Station,Bar
51,Hazrat Nizamuddin,2018-12-31,Underground,28.588749,77.257249,0,Train Station,Café,Moving Target,Food Court,Bus Station
52,Hindon,2019-03-08,Elevated,29.082006,77.458046,0,NaN,NaN,NaN,NaN,NaN
56,INA,2010-09-03,Underground,28.573669,77.207599,0,Market,Train Station,Light Rail Station,Restaurant,Yoga Studio


In [74]:
cluster1

,StationName,Opened,Layout,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Ashok Park Main,2010-04-02,Elevated,28.671633,77.155301,0,Train Station,Yoga Studio,Duty-free Shop,Food & Drink Shop,Food
8,Badarpur Border,2011-01-14,Elevated,28.493416,77.303334,0,ATM,Hotel,Train Station,Duty-free Shop,Food & Drink Shop
9,Bahadurgarh City,2018-06-24,Elevated,28.693324,76.933237,0,Bus Station,Train Station,Duty-free Shop,Food & Drink Shop,Food
30,Dwarka Sector 9,2006-04-01,Elevated,28.574282,77.065351,0,Herbs & Spices Store,Fast Food Restaurant,Train Station,Chaat Place,Burger Joint
43,Golf Course,2009-11-12,Elevated,28.597781,77.159817,0,Golf Course,Snack Place,Train Station,Sake Bar,Yoga Studio
44,Govind Puri,2010-10-03,Elevated,28.544377,77.264266,0,Beer Garden,Train Station,Pizza Place,Indian Restaurant,Park
50,Hauz Khas,2010-09-03,Underground,28.544258,77.206740,0,Lounge,Chinese Restaurant,Nightclub,Train Station,Bar
51,Hazrat Nizamuddin,2018-12-31,Underground,28.588749,77.257249,0,Train Station,Café,Moving Target,Food Court,Bus Station
52,Hindon,2019-03-08,Elevated,29.082006,77.458046,0,NaN,NaN,NaN,NaN,NaN
56,INA,2010-09-03,Underground,28.573669,77.207599,0,Market,Train Station,Light Rail Station,Restaurant,Yoga Studio
